# Notebook for exploring and clustering the Toronto neighbourhoods dataframe.

In [331]:
import requests
import pandas as pd
import numpy as np
import folium
import random
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

### Reading the neighbourhoods file.

In [332]:
df = pd.read_csv('Toronto_neighbourhoods_with_coords.csv')
df.head()

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [333]:
df.drop(["Unnamed: 0"], axis = 1, inplace = True)
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


### Creating a Toronto map with neighbourhoods location.

In [334]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent = "my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [335]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

 ### Setting the variables for the API calls.

In [336]:
CLIENT_ID = 'YW1G4YM4YVJ0BV3J0D2P1BXGESTULQYIWET3D1I5Z31Q4EA1'
CLIENT_SECRET = 'OI033BPVLR2ZEGO4MANA3DYGOM451QDTKNKZFIPLJZV1VZR5'
VERSION = '20180605'
LIMIT = 100

### Creating a function to get the neighbourhoods venues.

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Getting the venues.

In [8]:
toronto_venues = getNearbyVenues(names = df['Neighbourhood'],
                                   latitudes = df['Latitude'],
                                   longitudes = df['Longitude'])

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

In [9]:
toronto_venues.shape

(2258, 7)

In [10]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


### Counting the number of venues in each neighbourhood.

In [12]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",14,14,14,14,14,14
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
Berczy Park,55,55,55,55,55,55


### Finding the number of unique categories of venues.

In [13]:
print('{} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

276 unique categories.


### Analyzing the numbers of each venue type.

In [15]:
toronto_oneenc = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_oneenc['Neighbourhood'] = toronto_venues['Neighbourhood'] 

fixed_columns = [toronto_oneenc.columns[-1]] + list(toronto_oneenc.columns[:-1])
toronto_oneenc = toronto_oneenc[fixed_columns]

toronto_oneenc.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Grouping rows by neighbourhood and by taking the mean of the frequency of occurrence of each venue type.

In [23]:
toronto_mean_venues = toronto_oneenc.groupby('Neighbourhood').mean().reset_index()
toronto_mean_venues.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.01,0.0,0.000000,0.0,0.0,0.0,0.01,0.0,0.01,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.071429,0.0,0.0,0.0,0.00,0.0,0.00,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.0


### Printing each neighbourhood along with the top 5 most common venues on them.

In [24]:
num_top_venues = 5

for hood in toronto_mean_venues['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_mean_venues[toronto_mean_venues['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['Venue Type','Frequency']
    temp = temp.iloc[1:]
    temp['Frequency'] = temp['Frequency'].astype(float)
    temp = temp.round({'Frequency': 2})
    print(temp.sort_values('Frequency', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
            Venue Type  Frequency
0          Coffee Shop       0.06
1                 Café       0.05
2           Steakhouse       0.04
3  American Restaurant       0.04
4      Thai Restaurant       0.04


----Agincourt----
               Venue Type  Frequency
0  Furniture / Home Store       0.25
1          Breakfast Spot       0.25
2                  Lounge       0.25
3            Skating Rink       0.25
4       Accessories Store       0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
          Venue Type  Frequency
0         Playground        0.5
1               Park        0.5
2  Accessories Store        0.0
3  Mobile Phone Shop        0.0
4      Moving Target        0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
       Venue Type  Frequency
0   Grocery Store       0.14
1     Pizza Place       0.14
2     Coffee Shop       0.07
3  Sandwich Place       0.

### Translating the data above to a dataframe.

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [308]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_mean_venues['Neighbourhood']

for ind in np.arange(toronto_mean_venues.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_mean_venues.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Asian Restaurant,Cosmetics Shop,Bar,Hotel,Clothing Store
1,Agincourt,Furniture / Home Store,Lounge,Skating Rink,Breakfast Spot,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Pharmacy,Beer Store,Discount Store,Sandwich Place,Japanese Restaurant,Liquor Store,Video Store,Coffee Shop
4,"Alderwood, Long Branch",Pizza Place,Pharmacy,Skating Rink,Pub,Sandwich Place,Coffee Shop,Gym,Golf Course,Discount Store,Deli / Bodega
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Fried Chicken Joint,Frozen Yogurt Shop,Bridal Shop,Fast Food Restaurant,Sandwich Place,Middle Eastern Restaurant,Diner,Restaurant,Bank
6,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
7,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Hobby Shop,Indian Restaurant,Hardware Store,Japanese Restaurant,Juice Bar,Grocery Store
8,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Farmers Market,Cheese Shop,Café,Pub,Seafood Restaurant,Steakhouse,Bakery
9,"Birch Cliff, Cliffside West",College Stadium,Café,Skating Rink,General Entertainment,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant


# Clustering Neighbourhoods

#### Using k-means.

In [317]:
# number of clusters
kclusters = 5

toronto_clustering = toronto_mean_venues.drop('Neighbourhood', 1)

# k-means clustering
k_means = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

k_means.labels_[0:50] 

array([4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 0, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 0, 2, 4, 4,
       4, 0, 4, 4, 4, 4])

### Creating a dataframe with the clusters and the top ten neighbourhood venues.

In [318]:
toronto_m = df

toronto_m['Cluster Labels'] = pd.DataFrame(k_means.labels_)

toronto_m = toronto_m.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on = 'Neighbourhood')

toronto_m['Cluster Labels'] = toronto_m['Cluster Labels'].fillna(0.0).astype(int)

toronto_m.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4,Food & Drink Shop,Park,Bus Stop,Fast Food Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Falafel Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,4,Coffee Shop,Intersection,Hockey Arena,Portuguese Restaurant,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Café,Bakery,Park,Pub,Restaurant,Breakfast Spot,Mexican Restaurant,Theater,Shoe Store
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,4,Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Shoe Store,Miscellaneous Shop,Event Space,Women's Store,Coffee Shop,Vietnamese Restaurant
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,4,Coffee Shop,Gym,Sushi Restaurant,Japanese Restaurant,Diner,Yoga Studio,Portuguese Restaurant,Smoothie Shop,Seafood Restaurant,Boutique


### Visualizing a map with the clusters.

In [325]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_m['Latitude'], toronto_m['Longitude'], toronto_m['Neighbourhood'], toronto_m['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

### Examining the clusters.

#### Cluster 0 has a variety of venues, but has much more common physical activities related venues than any other cluster.

In [326]:
toronto_m.loc[toronto_m['Cluster Labels'] == 0, toronto_m.columns[[1] + list(range(5, toronto_m.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Café,Bakery,Park,Pub,Restaurant,Breakfast Spot,Mexican Restaurant,Theater,Shoe Store
13,North York,0,Gym,Grocery Store,Asian Restaurant,Coffee Shop,Beer Store,Bike Shop,Restaurant,Japanese Restaurant,Italian Restaurant,Sporting Goods Shop
16,York,0,Field,Hockey Arena,Trail,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
40,North York,0,Bus Stop,Park,Airport,Electronics Store,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
45,North York,0,Cafeteria,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Dim Sum Restaurant
58,Scarborough,0,College Stadium,Café,Skating Rink,General Entertainment,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
62,Central Toronto,0,Garden,Pool,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
65,Scarborough,0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Thrift / Vintage Store,Chinese Restaurant,Latin American Restaurant,Furniture / Home Store,Dumpling Restaurant,Eastern European Restaurant,Drugstore
67,Central Toronto,0,Park,Clothing Store,Dance Studio,Sandwich Place,Food & Drink Shop,Burger Joint,Hotel,Breakfast Spot,Grocery Store,Eastern European Restaurant
72,North York,0,Pharmacy,Pizza Place,Grocery Store,Coffee Shop,Butcher,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


#### Cluster 1 has as most common cafés, restaurants and coffee shops.

In [327]:
toronto_m.loc[toronto_m['Cluster Labels'] == 1, toronto_m.columns[[1] + list(range(5, toronto_m.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
80,Downtown Toronto,1,Café,Restaurant,Coffee Shop,Bar,Japanese Restaurant,Bookstore,Bakery,Chinese Restaurant,Poutine Place,College Arts Building


### Cluster 2 has coffee shops, restaurants and athletics & sports.

In [328]:
toronto_m.loc[toronto_m['Cluster Labels'] == 2, toronto_m.columns[[1] + list(range(5, toronto_m.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Scarborough,2,Athletics & Sports,Hakka Restaurant,Fried Chicken Joint,Bakery,Thai Restaurant,Caribbean Restaurant,Bank,Donut Shop,Dog Run,Doner Restaurant
36,Downtown Toronto,2,Coffee Shop,Aquarium,Hotel,Pizza Place,Café,Italian Restaurant,Brewery,Restaurant,Scenic Lookout,Bakery
41,East Toronto,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Pizza Place,Brewery,Bubble Tea Shop,Restaurant,Café
52,North York,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,North York,2,Coffee Shop,Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Hobby Shop,Indian Restaurant,Hardware Store,Japanese Restaurant,Juice Bar,Grocery Store


### Cluster 3 is the only one dominated by Breakfast Spots.

In [329]:
toronto_m.loc[toronto_m['Cluster Labels'] == 3, toronto_m.columns[[1] + list(range(5, toronto_m.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,West Toronto,3,Breakfast Spot,Gift Shop,Italian Restaurant,Cuban Restaurant,Bookstore,Movie Theater,Bank,Dessert Shop,Bar,Dog Run


### Cluster 4 has a wide variety of venues, and very common appearance of parks and coffee shops.

In [330]:
toronto_m.loc[toronto_m['Cluster Labels'] == 4, toronto_m.columns[[1] + list(range(5, toronto_m.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4,Food & Drink Shop,Park,Bus Stop,Fast Food Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Falafel Restaurant
1,North York,4,Coffee Shop,Intersection,Hockey Arena,Portuguese Restaurant,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant
3,North York,4,Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Shoe Store,Miscellaneous Shop,Event Space,Women's Store,Coffee Shop,Vietnamese Restaurant
4,Queen's Park,4,Coffee Shop,Gym,Sushi Restaurant,Japanese Restaurant,Diner,Yoga Studio,Portuguese Restaurant,Smoothie Shop,Seafood Restaurant,Boutique
5,Etobicoke,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Scarborough,4,Print Shop,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop
7,North York,4,Café,Basketball Court,Caribbean Restaurant,Baseball Field,Pool,Gym / Fitness Center,Japanese Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
8,East York,4,Fast Food Restaurant,Pizza Place,Pharmacy,Athletics & Sports,Gastropub,Intersection,Pet Store,Café,Rock Climbing Spot,Bank
9,Downtown Toronto,4,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Ramen Restaurant,Japanese Restaurant,Pizza Place,Plaza,Diner
10,North York,4,Japanese Restaurant,Pizza Place,Bakery,Playground,Pub,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


Notebook by Francisco Tosetto da Silva, made for the Capstone Project of the IBM Data Science Professional Certificate.